# Market Leader Analysis (PTP Obligation Bid Awards)

In [89]:
# Import necessary packages
import pandas as pd
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import numpy as np
import calendar
import glob
from datetime import datetime
from dateutil.parser import parse
import os
import urllib.request
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV


#Regression & LDA
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
%matplotlib inline

#see all columns/rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Import and Clean Data
###  strip/clean column headers, datetime conversion on:

##### Environmental: Market Data, Locational Marginal Pricing
*Third Party Environmental: NOAA Hourly Weather Data*
##### Transactional: Energy Only Offers/Awards
*Third Party Transactional: Daily NASDAQ, DOWJONES, ETF prices*

In [2]:
#import ERCOT
market_df = pd.read_excel('OneDrive_1_10-22-2019/ercot_market_data.xlsx', sheet_name = 'ercot_market_data')
lmp_df = pd.read_csv('OneDrive_1_10-22-2019/ercotlmp.csv')
nodes_df = pd.read_excel('OneDrive_1_10-22-2019/ercot_nodes.xlsx')
ptp_awards_df = pd.concat([pd.read_csv(f) for f in glob.glob('OneDrive_1_10-22-2019/PTPObligationBidAwards/*.csv')], ignore_index = True)

#strip/clean column headers
ptp_awards_df.columns = ptp_awards_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(' - ',' ')
ptp_awards_df = ptp_awards_df.rename(columns={'ptp_bid_award_-_mw':'ptp_bid_award_mw', 
                                                        'ptp_bid_-_price':'ptp_bid_price'})
market_df.columns = market_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
lmp_df.columns = lmp_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
nodes_df.columns = nodes_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')

#drop unnecessary columns
market_df = market_df.drop(columns=['datetime','year'])
lmp_df = lmp_df.drop(columns=['datetime','year'])
nodes_df = nodes_df.drop(columns=['iso','weatherstationid','first_dart_date','last_dart_date','equipment','voltage',
                                    'substation','nodetype','zoneid','objectid'])

#convert marketday feature from datetime type to string type for merging
market_df['marketday'] = market_df['marketday'].dt.strftime('%m/%d/%Y')

#import 3rd party data
weather_df_1 = pd.read_csv('additional_data/weather_data_1.csv')
weather_df_2 = pd.read_csv('additional_data/weather_data_2.csv')
weather_df_3 = pd.read_csv('additional_data/weather_data_3.csv')
nasdaq_df = pd.read_csv('additional_data/nasdaq_data.csv')
etf_df = pd.read_csv('additional_data/etf_data.csv')
dowjones_df = pd.read_csv('additional_data/dow_jones_data.csv')

#strip/clean column headers
weather_df_1.columns = weather_df_1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
weather_df_2.columns = weather_df_2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
weather_df_3.columns = weather_df_3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
nasdaq_df.columns = nasdaq_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
etf_df.columns = etf_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
dowjones_df.columns = dowjones_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')

#Select hourly weather data 
weather_df_1_trim = weather_df_1[['station','date','hourlydrybulbtemperature','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywinddirection','hourlywindspeed']]
weather_df_2_trim = weather_df_2[['station','date','hourlydrybulbtemperature','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywinddirection','hourlywindspeed']]
weather_df_3_trim = weather_df_3[['station','date','hourlydrybulbtemperature','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywinddirection','hourlywindspeed']]


#rename columns, drop 'close' column and use adjusted close column 'adj_close'
nasdaq_df = nasdaq_df.rename(columns={'date':'nasdaq_date','open':'nasdaq_open','high':'nasdaq_high',
                                     'low':'nasdaq_low','close':'nasdaq_close','adj_close':'nasdaq_adj_close','volume':'nasdaq_volume'})
nasdaq_df = nasdaq_df.drop(columns='nasdaq_close')
etf_df = etf_df.rename(columns={'date':'etf_date','open':'etf_open','high':'etf_high','low':'etf_low',
                                     'close':'etf_close','adj_close':'etf_adj_close','volume':'etf_volume'})
etf_df = etf_df.drop(columns='etf_close')
dowjones_df = dowjones_df.rename(columns={'date':'dowjones_date','open':'dowjones_open','high':'dowjones_high',
                                     'low':'dowjones_low','close':'dowjones_close','adj_close':'dowjones_adj_close','volume':'dowjones_volume'})
dowjones_df = dowjones_df.drop(columns='dowjones_close')

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,7,8,11,14,19,25,26,27,28,29,30,31,32,33,34,35,36,41,42,43,44,49,51,52,55,56,58,69,71,72,73,75,76,88,89,95,96,97,98,99,100,101,102,103,104,105,106,107,119,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,7,8,11,14,19,25,26,27,28,29,30,31,34,35,36,51,55,69,71,75,76,88,89,95,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31,32,34,35,36,42,44,49,55,56,69,71,72,75,76,88,89,95,96,97,98,99,100,101,102,103,104,105,106,107

### Displaying Content of ERCOT/ 3rd Party datasets

#### market_df

In [3]:
market_df.head()

,gr_panhandle_wind_stwpf,panhandle_ercot_rt_generic_constraints,ercot_total_resource_cap_out,ercot_gen_resource,ercot_rtload,ercot_original_load_forecast,ercot_wind_rti,ercot_wind_stwpf_orig,ercot_total_resource_cap_out.1,ercot_renew_resource_cap_out,hourending,marketday,peaktype,month
0,3713.4,3836,8525,50955.2,37081.44343,37031.792114,14317.82,13102.1,8525,1545,1,01/01/2019,OFFPEAK,JANUARY
1,3692.8,3836,8533,51139.2,37258.98993,37486.635803,14126.46,12025.1,8533,1588,2,01/01/2019,OFFPEAK,JANUARY
2,3587.2,3829,8533,51649.4,37300.18780,37394.205750,13686.93,11819.6,8533,1588,3,01/01/2019,OFFPEAK,JANUARY
3,3494.1,3826,8533,51116.7,37423.54347,37487.759094,13345.24,11499.3,8533,1569,4,01/01/2019,OFFPEAK,JANUARY
4,3426.3,3831,8533,50534.0,37895.21228,38759.001221,13238.17,11146.3,8533,1569,5,01/01/2019,OFFPEAK,JANUARY


#### nodes_df

In [4]:
nodes_df.head()

,nodename,zone,nearest_weatherstation
0,ACACIA_UNIT1,WEST,TX - Marfa/Municipal
1,AEEC,WEST,TX - Lubbock/Intl
2,AMISTAD_ALL,SOUTH,TX - San Angelo/Mathis
3,AMOCOOIL_CC1,HOUSTON,TX - Houston/Intercontinental
4,AMOCOOIL_CC2,HOUSTON,TX - Houston/Intercontinental


#### lmp_df

In [5]:
lmp_df.head()

,rtlmp,dalmp,hourending,marketday,peaktype,month,settlementpoint
0,13.0175,20.58,1,01/01/2019,OFFPEAK,JANUARY,AEEC
1,13.4000,14.48,1,01/01/2019,OFFPEAK,JANUARY,AMISTAD_ALL
2,14.2350,20.07,1,01/01/2019,OFFPEAK,JANUARY,AMOCOOIL_CC1
3,14.2350,20.07,1,01/01/2019,OFFPEAK,JANUARY,AMOCOOIL_CC2
4,14.2350,20.07,1,01/01/2019,OFFPEAK,JANUARY,AMOCO_PUN1


#### ptp_awards_df

In [6]:
ptp_awards_df.head()

,delivery_date,hour_ending,qse_name,settlement_point_source,settlement_point_sink,ptp_bid_award_mw,ptp_bid_price,bid_id
0,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,0.0,16.3,47079354
1,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,0.0,16.3,47079355
2,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,0.0,16.3,47079356
3,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,0.0,16.3,47079357
4,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,0.0,16.3,47079358


#### Example Weather: weather_df_1_trim

In [7]:
weather_df_1_trim.head()

,station,date,hourlydrybulbtemperature,hourlyrelativehumidity,hourlystationpressure,hourlywinddirection,hourlywindspeed
0,72267023042,2019-01-01T00:00:00,25,81.0,26.69,030,22
1,72267023042,2019-01-01T00:53:00,24,84.0,26.74,040,21
2,72267023042,2019-01-01T01:53:00,23,81.0,26.77,020,16
3,72267023042,2019-01-01T02:53:00,22,82.0,26.77,040,21
4,72267023042,2019-01-01T03:53:00,22,82.0,26.78,050,21


#### Example Stock: nasdaq_df

In [8]:
nasdaq_df.head()

,nasdaq_date,nasdaq_open,nasdaq_high,nasdaq_low,nasdaq_adj_close,nasdaq_volume
0,2019-01-02,6506.910156,6693.709961,6506.879883,6665.939941,2261800000
1,2019-01-03,6584.770020,6600.209961,6457.129883,6463.500000,2607290000
2,2019-01-04,6567.140137,6760.689941,6554.240234,6738.859863,2579550000
3,2019-01-07,6757.529785,6855.600098,6741.399902,6823.470215,2507550000
4,2019-01-08,6893.439941,6909.580078,6795.859863,6897.000000,2380290000


### Select Award Data for Top 10 Leaders in Timeframe 1.3.19 - 7.12.19

In [9]:
#Top 10 leaders
lead_ptp_awards = ptp_awards_df.loc[ptp_awards_df['qse_name'].isin(['QLUMN','QNRGTX','QDCENG','QREUEL','QSHELL',
                                                                    'QDIRE','QPREC','QMONT','QWOLFP','QTIOS'])]
#Select Data within timeline presented in Jeff's Power BI Dashboard (Jan 2, 2019 - July 12, 2019)
lead_ptp_awards['date'] = pd.to_datetime(lead_ptp_awards['delivery_date'])
mask = (lead_ptp_awards['date'] >= '2019-01-02') & (lead_ptp_awards['date'] < '2019-07-13')
lead_ptp_awards = lead_ptp_awards.loc[mask]

#lead_ptp_awards is the base of the merging section to create a model-ready dataset

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Join Market Data, LMP, & Leader Energy Award Tables = data

In [67]:
#Join Market & Location Pricing & Nodes Data
market_lmp = lmp_df.merge(market_df, how = 'left', on = ['marketday','hourending','peaktype','month'])
market_lmp_nodes = market_lmp.merge(nodes_df, how='left', left_on='settlementpoint', right_on='nodename') #merge with nodes
market_lmp_nodes = market_lmp_nodes.drop(columns='nodename') #drop unnecessary columns
#Join PTP Awards with joined Market/Price df
lead_ptp_awards = lead_ptp_awards.rename(columns={'delivery_date':'marketday','hour_ending':'hourending',
                                                  'settlement_point_source':'settlementpoint_src','settlement_point_sink':'settlementpoint_snk'})
awards_Src = lead_ptp_awards.merge(market_lmp_nodes.add_prefix('src_'), how = 'left', 
                                          left_on = ['marketday','hourending','settlementpoint_src'],
                                          right_on = ['src_marketday','src_hourending','src_settlementpoint'])
data_ercot = awards_Src.merge(market_lmp_nodes.add_prefix('snk_'), how = 'left', 
                                          left_on = ['marketday','hourending','settlementpoint_snk'],
                                          right_on = ['snk_marketday','snk_hourending','snk_settlementpoint'])
data_ercot.loc[(data_ercot.snk_nearest_weatherstation == 'TM - Nuevo Laredo/Intl'),'snk_nearest_weatherstation']='TX - Laredo/Intl'
data_ercot.loc[(data_ercot.src_nearest_weatherstation == 'TM - Nuevo Laredo/Intl'),'src_nearest_weatherstation']='TX - Laredo/Intl'
data_ercot = data_ercot.fillna(0.0)
data_ercot.loc[(data_ercot.snk_zone == 0.0),'snk_zone']='Other'

In [69]:
lead_ptp_awards.shape

(4856190, 9)

In [70]:
data_ercot.shape

(4856190, 47)

In [71]:
stocks_df = nasdaq_df.merge(etf_df, how='left',left_on='nasdaq_date',right_on='etf_date')
stocks_df = stocks_df.merge(dowjones_df, how='left',left_on='nasdaq_date',right_on='dowjones_date')
stocks_df = stocks_df.drop(columns=['dowjones_date','etf_date'])
stocks_df = stocks_df.rename(columns={'nasdaq_date':'date'})
stocks_df['date'] = pd.to_datetime(stocks_df['date'],infer_datetime_format=True)
stocks_df['date'] = stocks_df['date'].dt.strftime('%m/%d/%Y')

In [72]:
weather_df = pd.concat([weather_df_1_trim, weather_df_2_trim,weather_df_3_trim])
weather_df['hourlydrybulbtemperature'] = pd.to_numeric(weather_df['hourlydrybulbtemperature'], errors='coerce', downcast=None)
weather_df['hourlystationpressure'] = pd.to_numeric(weather_df['hourlystationpressure'], errors='coerce', downcast=None)
weather_df['hourlywinddirection'] = pd.to_numeric(weather_df['hourlywinddirection'], errors='coerce', downcast=None)
weather_df['hourlywindspeed'] = pd.to_numeric(weather_df['hourlywindspeed'], errors='coerce', downcast=None)

weather_df['date'] = pd.to_datetime(weather_df['date'],infer_datetime_format=True)
weather_df['hourending'] = [d.time() for d in weather_df['date']]
mask = (weather_df['date'] >= '2019-01-02') & (weather_df['date'] < '2019-07-13')
weather_df = weather_df.loc[mask]
hours = [math.ceil((t.hour * 60 + t.minute) / 60) for t in weather_df['hourending']]
weather_df['hour'] = hours
weather_df['hour']= weather_df['hour'].apply(str).apply(int)
weather_df = weather_df.loc[(weather_df['hour'] > 0)]
weather_df['date'] = weather_df['date'].dt.strftime('%m/%d/%Y')
c_maxes = weather_df.groupby(['station', 'date','hour']).hourending.transform(max)
weather_df = weather_df.loc[weather_df.hourending == c_maxes]
weather_df['station'] = weather_df['station'].map({72267023042: 'TX - Lubbock/Intl',
                                                                 72251012924: 'TX - Corpus Christi/Intl',
                                                                 72266013962: 'TX - Abilene/Municipal', 
                                                                 72250012919: 'TX - Brownsville/Intl', 
                                                                 72351013966: 'TX - Wichita Falls/Sheppard AFB',
                                                                 72261022010: 'TX - Del Rio/Intl',
                                                                 72265023023: 'TX - Midland-Odessa',
                                                                 72253012921: 'TX - San Antonio/Intl',
                                                                 72363023047: 'TX - Amarillo/Intl',
                                                                 72248013957: 'LA - Shreveport/Regional',
                                                                 72263023034: 'TX - San Angelo/Mathis',
                                                                 72265623040: 'TX - Wink/Winkler County',
                                                                 72258013960: 'TX - Dallas/Love Field',
                                                                 72243012960: 'TX - Houston/Intercontinental',
                                                                 72261823091: 'TX - Fort Stockton',
                                                                 72252012907: 'TX - Laredo/Intl',
                                                                 74641013975: 'OK - Gage/Shattuck',
                                                                 72259303985: 'TX - Dallas-Fort Worth/Intl'})

In [73]:
weather_df=weather_df.drop_duplicates(keep='first')
data = data_ercot.merge(weather_df.drop_duplicates(['date','hour','station']).add_prefix('src_'), 
                       how = 'left', 
                       left_on = ['marketday','hourending','src_nearest_weatherstation'], 
                       right_on=['src_date','src_hour','src_station'])
data = data.merge(weather_df.drop_duplicates(['date','hour','station']).add_prefix('snk_'), 
                       how = 'left', 
                       left_on = ['marketday','hourending','snk_nearest_weatherstation'], 
                       right_on=['snk_date','snk_hour','snk_station'])

In [74]:
data = data.merge(stocks_df, how = 'left', left_on='marketday', right_on='date')
data = data.fillna(0.00)
data.loc[(data.src_zone == 0.0),'src_zone']='Other'

In [75]:
data_ercot.shape

(4856190, 47)

In [76]:
data.shape

(4856190, 81)

### Create Evaluation Criterion: PnL

In [77]:
#Create PnL column for Performance Measurement/Evaluation Criterion
data['PnL'] = ((data.src_dalmp - data.snk_dalmp) - (data.src_rtlmp - data.snk_rtlmp)) * data.ptp_bid_award_mw
#Separate Leaders into dataframes for separate modeling
leaders_QLUMN = data.loc[data['qse_name'] == 'QLUMN']
leaders_QNRGTX = data.loc[data['qse_name'] == 'QNRGTX']
leaders_QDCENG = data.loc[data['qse_name'] == 'QDCENG']
leaders_QREUEL = data.loc[data['qse_name'] == 'QREUEL']
leaders_QSHELL = data.loc[data['qse_name'] == 'QSHELL']
leaders_QDIRE = data.loc[data['qse_name'] == 'QDIRE']
leaders_QPREC = data.loc[data['qse_name'] == 'QPREC']
leaders_QMONT = data.loc[data['qse_name'] == 'QMONT']
leaders_QWOLFP = data.loc[data['qse_name'] == 'QWOLFP']
leaders_QTIOS = data.loc[data['qse_name'] == 'QTIOS']


In [60]:
leaders_QLUMN['PnL'].sum() #PnL: 4,938,417.14
leaders_QNRGTX['PnL'].sum() #2,245,426.25
leaders_QDCENG['PnL'].sum() #1,370,061.43
leaders_QREUEL['PnL'].sum() #1,344,345.49
leaders_QSHELL['PnL'].sum() #1,322,784.03
leaders_QDIRE['PnL'].sum() #1,228,761.44
leaders_QPREC['PnL'].sum() #1,024,767.48
leaders_QMONT['PnL'].sum() #979,167.64
leaders_QWOLFP['PnL'].sum() #958,999.75
PnL = leaders_QTIOS['PnL'].sum() #790,331.15
PnL 


790331.1544999998

# Model Test 1: Linear Least Squares Regression

In [78]:
leader = leaders_QLUMN
leader =leader.drop(columns=['marketday','qse_name','ptp_bid_price','bid_id','date_x','src_hourending_x','src_settlementpoint','snk_hourending_x','snk_month','snk_settlementpoint','src_station','src_hourending_y','snk_station','snk_hourending_y','date_y','src_marketday','snk_marketday','src_date','snk_date','src_hour','snk_hour', 'src_nearest_weatherstation','snk_nearest_weatherstation','snk_ercot_total_resource_cap_out.1','snk_rtlmp','src_rtlmp'])
leader = leader.fillna(0.00)


In [79]:
x = leader.iloc[:,:-1].values
y =leader.iloc[:,55].values
labelencoder_x=LabelEncoder()
x[:,1]=labelencoder_x.fit_transform(x[:,1])
x[:,2]=labelencoder_x.fit_transform(x[:,2])
x[:,5]=labelencoder_x.fit_transform(x[:,5])
x[:,6]=labelencoder_x.fit_transform(x[:,6])
x[:,17]=labelencoder_x.fit_transform(x[:,17])
x[:,19]=labelencoder_x.fit_transform(x[:,19])
x[:,29]=labelencoder_x.fit_transform(x[:,29])


#onehotencoder=OneHotEncoder(categorical_features =[1])
#x =onehotencoder.fit_transform(x).toarray()


In [80]:
#splitting training set and testing set
xtrain, xtest, ytrain, ytest =train_test_split(x,y,test_size=0.2)

# Training the Multivariate Linear Regression Model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(xtrain, ytrain)

# Predicting the Test set results
y_prediction= regressor.predict(xtest)

# Backward Eliminiation

# Insert B Intercept
X=np.append(arr = np.ones((259246,1)).astype(int), values = x, axis = 1)

In [82]:
# Call Ordinary Least Square
xelimination = X[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]
xelimination = np.array(xelimination, dtype=float)
regressorOLS = smf.OLS(y, xelimination).fit()
regressorOLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     41.93
Date:                Fri, 08 Nov 2019   Prob (F-statistic):          2.29e-148
Time:                        17:03:30   Log-Likelihood:            -2.3084e+06
No. Observations:              259246   AIC:                         4.617e+06
Df Residuals:                  259227   BIC:                         4.617e+06
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -245.7651     83.217     -2.953      0.003    -408.867     -82.663
x1            -0.3767      0.631     -0.597      0.551      -1.613       0.860
x2            -5.2854      0.899     -5.881      0.000      -7.047      -3.524
x3            -7.3497      1.598     -4.600      0.000     -10.481      -4.218
x4             0.1402      0.023      6.193      0.000       0.096       0.185
x5            -7.3752      1.421     -5.191      0.000     -10.160      -4.591
x6             0.5707      2.809      0.203      0.839      -4.934       6.076
x7             2.5496      2.339      1.090      0.276      -2.035       7.134
x8             0.0146      0.002      6.384      0.000       0.010       0.019
x9            -0.0066      0.007     -0.938      0.348      -0.020       0.007
x10            0.0030      0.000      9.773      0.000       0.002       0.004
x11            0.0024      0.001      2.761      0.006       0.001       0.004
x12            0.0024      0.001      2.273      0.023       0.000       0.005
x13           -0.0033      0.001     -2.907      0.004      -0.005      -0.001
x14           -0.0223      0.001    -17.687      0.000      -0.025      -0.020
x15            0.0217      0.001     18.138      0.000       0.019       0.024
x16            0.0030      0.000      9.773      0.000       0.002       0.004
x17            0.0017      0.004      0.415      0.678      -0.007       0.010
x18            8.8290      6.409      1.378      0.168      -3.732      21.390
x19            6.9594      1.402      4.963      0.000       4.211       9.708
x20            0.5707      2.809      0.203      0.839      -4.934       6.076
x21            0.0146      0.002      6.384      0.000       0.010       0.019
x22           -0.0066      0.007     -0.938      0.348      -0.020       0.007
x23            0.0030      0.000      9.773      0.000       0.002       0.004
x24            0.0024      0.001      2.761      0.006       0.001       0.004
x25            0.0024      0.001      2.273      0.023       0.000       0.005
x26           -0.0033      0.001     -2.907      0.004      -0.005      -0.001
x27           -0.0223      0.001    -17.687      0.000      -0.025      -0.020
x28            0.0217      0.001     18.138      0.000       0.019       0.024
x29            0.0017      0.004      0.415      0.678      -0.007       0.010
==============================================================================
Omnibus:                   704959.837   Durbin-Watson:                   1.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     120114179255.895
Skew:                          33.100   Prob(JB):                         0.00
Kurtosis:                    3336.966   Cond. No.                     2.07e+16
==============================================================================

W

In [88]:
# Call Ordinary Least Square
xelimination = X[:,[0,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54]]
xelimination = np.array(xelimination, dtype=float)
regressorOLS = smf.OLS(y, xelimination).fit()
regressorOLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     22.82
Date:                Fri, 08 Nov 2019   Prob (F-statistic):          2.27e-104
Time:                        17:20:05   Log-Likelihood:            -2.3085e+06
No. Observations:              259246   AIC:                         4.617e+06
Df Residuals:                  259220   BIC:                         4.617e+06
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -15.2521     18.595     -0.820      0.412     -51.697      21.193
x1             3.1140      3.005      1.036      0.300      -2.776       9.004
x2            -0.6092      0.720     -0.847      0.397      -2.020       0.801
x3             0.7429      0.347      2.138      0.032       0.062       1.424
x4            -4.1912      2.426     -1.728      0.084      -8.945       0.563
x5             0.0169      0.046      0.368      0.713      -0.073       0.107
x6             5.6891      1.075      5.291      0.000       3.582       7.796
x7            -0.0055      0.699     -0.008      0.994      -1.376       1.365
x8             1.5403      0.349      4.410      0.000       0.856       2.225
x9             0.0247      2.318      0.011      0.991      -4.518       4.568
x10           -0.0770      0.047     -1.621      0.105      -0.170       0.016
x11           -3.2444      1.082     -2.999      0.003      -5.365      -1.124
x12           -2.0042      0.216     -9.271      0.000      -2.428      -1.581
x13            2.3680      0.308      7.700      0.000       1.765       2.971
x14            1.3529      0.298      4.546      0.000       0.770       1.936
x15           -1.5759      0.310     -5.091      0.000      -2.183      -0.969
x16         4.296e-08   1.01e-08      4.268      0.000    2.32e-08    6.27e-08
x17          -17.1711     13.229     -1.298      0.194     -43.099       8.757
x18          -93.3000     17.742     -5.259      0.000    -128.074     -58.526
x19          106.9705     15.654      6.833      0.000      76.288     137.653
x20           18.1035     12.377      1.463      0.144      -6.154      42.361
x21           -0.0002   2.66e-05     -7.689      0.000      -0.000      -0.000
x22            0.5706      0.080      7.092      0.000       0.413       0.728
x23           -0.6726      0.117     -5.730      0.000      -0.903      -0.443
x24           -0.2945      0.099     -2.982      0.003      -0.488      -0.101
x25            0.3124      0.111      2.811      0.005       0.095       0.530
==============================================================================
Omnibus:                   706087.187   Durbin-Watson:                   1.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     120723918418.885
Skew:                          33.245   Prob(JB):                         0.00
Kurtosis:                    3345.415   Cond. No.                     1.14e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.14e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [92]:
# Call Ordinary Least Square
xelimination = X[:,[0,8,9,10,11,12,13,14,15,16,17,22,23,24,25,26,27,28,29,34,36,37,45,50]]
xelimination = np.array(xelimination, dtype=float)
regressorOLS = smf.OLS(y, xelimination).fit()
regressorOLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     57.36
Date:                Fri, 08 Nov 2019   Prob (F-statistic):          4.54e-162
Time:                        17:52:50   Log-Likelihood:            -2.3084e+06
No. Observations:              259246   AIC:                         4.617e+06
Df Residuals:                  259231   BIC:                         4.617e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -395.5687     88.446     -4.472      0.000    -568.920    -222.218
x1             0.0335      0.005      7.327      0.000       0.025       0.043
x2            -0.0118      0.008     -1.533      0.125      -0.027       0.003
x3             0.0033      0.000     10.964      0.000       0.003       0.004
x4             0.0029      0.001      3.517      0.000       0.001       0.005
x5             0.0026      0.001      2.457      0.014       0.001       0.005
x6            -0.0029      0.001     -2.629      0.009      -0.005      -0.001
x7            -0.0219      0.001    -17.447      0.000      -0.024      -0.019
x8             0.0211      0.001     17.713      0.000       0.019       0.023
x9             0.0033      0.000     10.964      0.000       0.003       0.004
x10           -0.0081      0.004     -1.936      0.053      -0.016       0.000
x11           -0.0118      0.008     -1.533      0.125      -0.027       0.003
x12            0.0033      0.000     10.964      0.000       0.003       0.004
x13            0.0029      0.001      3.517      0.000       0.001       0.005
x14            0.0026      0.001      2.457      0.014       0.001       0.005
x15           -0.0029      0.001     -2.629      0.009      -0.005      -0.001
x16           -0.0219      0.001    -17.447      0.000      -0.024      -0.019
x17            0.0211      0.001     17.713      0.000       0.019       0.023
x18           -0.0081      0.004     -1.936      0.053      -0.016       0.000
x19           -0.0122      0.034     -0.355      0.723      -0.079       0.055
x20           -0.8347      0.230     -3.637      0.000      -1.285      -0.385
x21            1.6643      0.184      9.056      0.000       1.304       2.024
x22         3.967e-08   4.56e-09      8.703      0.000    3.07e-08    4.86e-08
x23           -0.0002   2.37e-05     -8.902      0.000      -0.000      -0.000
==============================================================================
Omnibus:                   705757.349   Durbin-Watson:                   1.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     120819640312.043
Skew:                          33.201   Prob(JB):                         0.00
Kurtosis:                    3346.742   Cond. No.                     4.24e+22
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.04e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [91]:
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-4,1e-3,1e-2,1,5,10,20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error',cv=20)
lasso_regressor.fit(xtrain,ytrain)
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceW

{'alpha': 1}
-3125154.613872563


/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [83]:

lasso = Lasso()
lasso.fit(xtrain,ytrain)
train_score=lasso.score(xtrain,ytrain)
test_score=lasso.score(xtest,ytest)
coeff_used = np.sum(lasso.coef_!=0)

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [84]:
print( "training score:", train_score )
print( "test score: ", test_score)
print ("number of features used: ", coeff_used)

training score: 0.003958540245053377
test score:  0.0036012144927155543
number of features used:  52


In [85]:
lasso001 = Lasso(alpha=0.01, max_iter=10e2)
lasso001.fit(xtrain,ytrain)
train_score001=lasso001.score(xtrain,ytrain)
test_score001=lasso001.score(xtest,ytest)
coeff_used001 = np.sum(lasso001.coef_!=0)
print ("training score for alpha=0.01:", train_score001 )
print ("test score for alpha =0.01: ", test_score001)
print ("number of features used: for alpha =0.01:", coeff_used001)


/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


training score for alpha=0.01: 0.003961652722870412
test score for alpha =0.01:  0.0036106960299957525
number of features used: for alpha =0.01: 55


In [86]:
lasso00001 = Lasso(alpha=0.0001, max_iter=10e2)
lasso00001.fit(xtrain,ytrain)
train_score00001=lasso00001.score(xtrain,ytrain)
test_score00001=lasso00001.score(xtest,ytest)
coeff_used00001 = np.sum(lasso00001.coef_!=0)
print ("training score for alpha=0.0001:", train_score00001 )
print ("test score for alpha =0.0001: ", test_score00001)
print ("number of features used: for alpha =0.0001:", coeff_used00001)

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


training score for alpha=0.0001: 0.003961590392129621
test score for alpha =0.0001:  0.0036106852912333176
number of features used: for alpha =0.0001: 55
